# Colab-deep-daze
Original repo: [lucidrains/deep-daze](https://github.com/lucidrains/deep-daze)

My fork: [styler00dollar/Colab-deep-daze](https://github.com/styler00dollar/Colab-deep-daze)

In [ ]:
!nvidia-smi

In [ ]:
#@title Install
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install deep-daze

# Using Google Drive and disabling heavy printing to avoid output size limit

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')
!mkdir '/content/drive/MyDrive/DeepDaze/'

Current default is ```!mkdir '/content/drive/MyDrive/DeepDaze/'```

In [ ]:
from tqdm import tqdm
from IPython.display import Image, display
import shutil

from deep_daze import Imagine

save_path = '/content/drive/MyDrive/DeepDaze/' #@param {type:"string"}
TEXT = 'happy anime character' #@param {type:"string"}
TEXT_PATH = TEXT.replace(" ", "_")
NUM_LAYERS = 32 #@param {type:"number"}

model = Imagine(
    text = TEXT,
    num_layers = NUM_LAYERS,
    save_every = 50 #@param {type:"number"}
)

# without heavy printing and saving images to drive
for epoch in range(20):
    for i in range(1000):
        model.train_step(epoch, i)

        if i % model.save_every != 0:
            continue

        file_end = str("_"+str({epoch})+"_"+str({i})+".png")
        source = f'{TEXT_PATH}.png'
        destination = f'{save_path}{TEXT_PATH}{file_end}'
        print(source, destination)
        shutil.move(source, destination)


# Original with trange

Warning: In this current state you will get ```Buffered data was truncated after reaching the output size limit.``` after a while and thus dont see further progress. Combined with Colabs terminating behaviour and risking to loose the output with missing printing, this is not really a good idea to use.

In [ ]:
from tqdm import trange
from IPython.display import Image, display

from deep_daze import Imagine

TEXT = 'an apple next to a fireplace' #@param {type:"string"}
NUM_LAYERS = 32 #@param {type:"number"}

model = Imagine(
    text = TEXT,
    num_layers = NUM_LAYERS,
    save_every = 50
)

for epoch in trange(20, desc = 'epochs'):
    for i in trange(1000, desc = 'iteration'):
        model.train_step(epoch, i)

        if i % model.save_every != 0:
            continue

        image = Image(f'./{TEXT}.png')
        display(image)